# Workflow for a multi-regional energy system

In this application of the FINE framework, a multi-regional energy system is modeled and optimized.

All classes which are available to the user are utilized and examples of the selection of different parameters within these classes are given.

The workflow is structures as follows:
1. Required packages are imported and the input data path is set
2. An energy system model instance is created
3. Commodity sources are added to the energy system model
4. Commodity conversion components are added to the energy system model
5. Commodity storages are added to the energy system model
6. Commodity transmission components are added to the energy system model
7. Commodity sinks are added to the energy system model
8. The energy system model is optimized
9. Selected optimization results are presented


# 1. Import required packages and set input data path

The FINE framework is imported which provides the required classes and functions for modeling the energy system.

In [1]:
import FINE as fn
from getData import getData
data = getData()

%load_ext autoreload
%autoreload 2

# 2. Create an energy system model instance 

The structure of the energy system model is given by the considered locations, commodities, the number of time steps as well as the hours per time step.

The commodities are specified by a unit (i.e. 'GW_electric', 'GW_H2lowerHeatingValue', 'Mio. t CO2/h') which can be given as an energy or mass unit per hour. Furthermore, the cost unit and length unit are specified.

In [2]:
locations = {'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6', 'cluster_7'}
commodityUnitDict = {'electricity': 'GW_el', 'methane': 'GW_CH4_LHV', 'biogas': 'GW_biogas_LHV',
                     'CO2': 'Mio. t CO2/h', 'hydrogen': 'GW_H2_LHV'}
commodities = {'electricity', 'hydrogen', 'methane', 'biogas', 'CO2'}
numberOfTimeSteps=8760
hoursPerTimeStep=1

In [3]:
esM = fn.EnergySystemModel(locations=locations, commodities=commodities, numberOfTimeSteps=8760,
                           commoditiyUnitsDict=commodityUnitDict,
                           hoursPerTimeStep=1, costUnit='1e9 Euro', lengthUnit='km')

In [4]:
CO2_reductionTarget = 0.9

# 3. Add commodity sources to the energy system model

## 3.1. Electricity sources

### Wind onshore

In [5]:
esM.add(fn.Source(esM=esM, name='Wind (onshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (onshore), operationRateMax'],
                  capacityMax=data['Wind (onshore), capacityMax'],
                  investPerCapacity=1.1, opexPerCapacity=1.1*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [6]:
data['Wind (onshore), operationRateMax'].sum()

cluster_0    2300.406907
cluster_1    1082.032187
cluster_2    1447.812026
cluster_3    1448.827987
cluster_4    1935.618218
cluster_5    1757.696047
cluster_6    1286.944636
cluster_7    1199.380071
dtype: float64

### Wind offshore

In [7]:
esM.add(fn.Source(esM=esM, name='Wind (offshore)', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['Wind (offshore), operationRateMax'],
                  capacityMax=data['Wind (offshore), capacityMax'],
                  investPerCapacity=2.3, opexPerCapacity=2.3*0.02, interestRate=0.08,
                  economicLifetime=20))

Full load hours:

In [8]:
data['Wind (offshore), operationRateMax'].sum()

cluster_0    4332.977326
cluster_1       0.000000
cluster_2       0.000000
cluster_3       0.000000
cluster_4    3963.044612
cluster_5    3459.283279
cluster_6       0.000000
cluster_7       0.000000
dtype: float64

### PV

In [9]:
esM.add(fn.Source(esM=esM, name='PV', commodity='electricity', hasCapacityVariable=True,
                  operationRateMax=data['PV, operationRateMax'], capacityMax=data['PV, capacityMax'],
                  investPerCapacity=0.65, opexPerCapacity=0.65*0.02, interestRate=0.08,
                  economicLifetime=25))

Full load hours:

In [10]:
data['PV, operationRateMax'].sum()

cluster_0    859.175855
cluster_1    970.535848
cluster_2    913.049274
cluster_3    917.880212
cluster_4    875.448945
cluster_5    904.590003
cluster_6    984.223517
cluster_7    992.311727
dtype: float64

### Exisisting run-of-river hydroelectricity plants

In [11]:
esM.add(fn.Source(esM=esM, name='Existing run-of-river plants', commodity='electricity',
                  hasCapacityVariable=True,
                  operationRateFix=data['Existing run-of-river plants, operationRateFix'],
                  capacityFix=data['Existing run-of-river plants, capacityFix'],
                  investPerCapacity=0, opexPerCapacity=0.208))

## 3.2. Methane (natural gas and biogas)

### Natural gas

In [12]:
esM.add(fn.Source(esM=esM, name='Natural gas purchase', commodity='methane',
                  hasCapacityVariable=False, commodityCost=0.0331*1e-3))

### Biogas

In [13]:
esM.add(fn.Source(esM=esM, name='Biogas purchase', commodity='biogas',
                  operationRateMax=data['Biogas, operationRateMax'], hasCapacityVariable=False,
                  commodityCost=0.05409*1e-3))

## 3.3 CO2

### CO2

The CO2 source is required for the case in which it can be mixed with conventional natural gas.

In [14]:
esM.add(fn.Source(esM=esM, name='CO2 from enviroment', commodity='CO2',
                  hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 4. Add conversion components to the energy system model

### Biogas to methane

In [15]:
esM.add(fn.Conversion(esM=esM, name='Biogas to methane', physicalUnit='GW_biogas_LHV',
                      commodityConversionFactors={'biogas':-1, 'methane':1, 'CO2':-201*1e-6*0.625},
                      hasCapacityVariable=False))

### Existing combined cycle gas turbine plants

In [16]:
esM.add(fn.Conversion(esM=esM, name='Existing CCGT plants (methane)', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':1, 'methane':-1/0.625, 'CO2':201*1e-6/0.625},
                      hasCapacityVariable=True, capacityMax=data['Existing CCGT plants (methane), capacityMax'],
                      investPerCapacity=0, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycle gas turbine plants for biogas

In [17]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (biogas)', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':1, 'biogas':-1/0.635},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.7, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### New combined cycly gas turbines for hydrogen

In [18]:
esM.add(fn.Conversion(esM=esM, name='New CCGT plants (hydrogen)', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':1, 'hydrogen':-1/0.6},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.927, opexPerCapacity=0.021, interestRate=0.08,
                      economicLifetime=33))

### Electrolyzers

In [19]:
esM.add(fn.Conversion(esM=esM, name='Electroylzers', physicalUnit='GW_el',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=0.5, opexPerCapacity=0.5*0.025, interestRate=0.08,
                      economicLifetime=10))

# 5. Add commodity storages to the energy system model

## 5.1. Electricity storage

### Lithium ion batteries

The self discharge of a lithium ion battery is here described as 3% per month. The self discharge per hours is obtained using the equation (1-$\text{selfDischarge}_\text{hour})^{30*24\text{h}} = 1-\text{selfDischarge}_\text{month}$.

In [20]:
esM.add(fn.Storage(esM=esM, name='Li-ion batteries', commodity='electricity',
                   hasCapacityVariable=True, chargeEfficiency=0.95,
                   cyclicLifetime=10000, dischargeEfficiency=0.95, selfDischarge=1-(1-0.03)**(1/(30*24)),
                   chargeRate=1, dischargeRate=1,
                   investPerCapacity=0.151, opexPerCapacity=0.002, interestRate=0.08,
                   economicLifetime=22))

## 5.2. Hydrogen storage

### Hydrogen filled salt caverns
The maximum capacity is here obtained by: dividing the given capacity (which is given for methane) by the lower heating value of methane and then multiplying it with the lower heating value of hydrogen.

In [21]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (hydrogen)', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=133,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (hydrogen), capacityMax'],
                   investPerCapacity=0.00011, opexPerCapacity=0.00057, interestRate=0.08,
                   economicLifetime=30))

## 5.3. Methane storage

### Methane filled salt caverns

In [22]:
esM.add(fn.Storage(esM=esM, name='Salt caverns (methane)', commodity='methane',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   capacityPerPlantUnit=443,
                   chargeRate=1/470.37, dischargeRate=1/470.37, sharedPotentialID='Existing salt caverns',
                   stateOfChargeMin=0.33, stateOfChargeMax=1, capacityMax=data['Salt caverns (methane), capacityMax'],
                   investPerCapacity=0.00004, opexPerCapacity=0.00001, interestRate=0.08,
                   economicLifetime=30))

## 5.4 Pumped hydro storage

### Pumped hydro storage

In [23]:
esM.add(fn.Storage(esM=esM, name='Pumped hydro storage', commodity='electricity',
                   chargeEfficiency=0.88, dischargeEfficiency=0.88,
                   hasCapacityVariable=True, selfDischarge=1-(1-0.00375)**(1/(30*24)),
                   chargeRate=0.16, dischargeRate=0.12, capacityFix=data['Pumped hydro storage, capacityFix'],
                   investPerCapacity=0, opexPerCapacity=0.000153))

# 6. Add commodity transmission components to the energy system model

## 6.1. Electricity transmission

### AC cables

In [24]:
esM.add(fn.Transmission(esM=esM, name='AC cables', commodity='electricity',
                        hasCapacityVariable=True, capacityFix=data['AC cables, capacityFix']))

The distances of a component are set to a normalized values of 1.


### DC cables

In [25]:
esM.add(fn.Transmission(esM=esM, name='DC cables', commodity='electricity', losses=data['DC cables, losses'],
                        distances=data['DC cables, distances'],
                        hasCapacityVariable=True, capacityFix=data['DC cables, capacityFix']))

## 6.2 Methane transmission

### Methane pipeline

In [26]:
esM.add(fn.Transmission(esM=esM, name='Pipelines (methane)', commodity='methane',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        investPerCapacity=0.000037, investIfBuilt=0.000314,
                        interestRate=0.08, economicLifetime=40))

## 6.3 Hydrogen transmission

### Hydrogen pipelines

In [27]:
esM.add(fn.Transmission(esM=esM, name='Pipeline (hydrogen)', commodity='hydrogen',
                        distances=data['Pipelines, distances'],
                        hasCapacityVariable=True, hasIsBuiltBinaryVariable=True, bigM=300,
                        locationalEligibility=data['Pipelines, eligibility'],
                        investPerCapacity=0.000177, investIfBuilt=0.00033,
                        interestRate=0.08, economicLifetime=40))

# 7. Add commodity sinks to the energy system model

## 7.1. Electricity sinks

### Electricity demand

In [28]:
esM.add(fn.Sink(esM=esM, name='Electricity demand', commodity='electricity',
                hasCapacityVariable=False, operationRateFix=data['Electricity demand, operationRateFix']))

## 7.2. Hydrogen sinks

### Fuel cell electric vehicle (FCEV) demand

In [29]:
FCEV_penetration=0.5
esM.add(fn.Sink(esM=esM, name='Hydrogen demand', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix=data['Hydrogen demand, operationRateFix']*FCEV_penetration))

## 7.3. CO2 sinks

### CO2 exiting the system's boundary

In [30]:
esM.add(fn.Sink(esM=esM, name='CO2 to enviroment', commodity='CO2',
                hasCapacityVariable=False, commodityLimitID='CO2 limit', yearlyLimit=366*(1-CO2_reductionTarget)))

# 8. Optimize energy system model

All components are now added to the model and the model can be optimized. If the computational complexity of the optimization should be reduced, the time series data of the specified components can be clustered before the optimization and the parameter timeSeriesAggregation is set to True in the optimize call.

In [31]:
esM.cluster(numberOfTypicalPeriods=7)


Clustering time series data with 7 typical periods and 24 time steps per period...
		(3.8580 sec)



In [32]:
esM.optimize(timeSeriesAggregation=True, optimizationSpecs='LogToConsole=1 OptimalityTol=1e-3 cuts=0 method=2')

Time series aggregation specifications:
Number of typical periods: 7 , number of time steps per periods: 24
Declaring sets, variables and constraints for ConversionModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.1770 sec)
Declaring sets, variables and constraints for StorageModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(1.4270 sec)
Declaring sets, variables and constraints for SourceSinkModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.4550 sec)
Declaring sets, variables and constraints for TransmissionModeling
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.3090 sec)
Declaring shared potential constraint...
		(0.0010 sec)
Declaring commodity balances...
		(0.9860 sec)
Declaring objective function...
		(0.5460 sec)
Academic license - for non-commercial use only
Changed value of parameter QCPDual to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Parameter

# 9. Selected results output

### Sources and Sink

In [33]:
esM.getOptimizationSummary("SourceSinkModeling", outputLevel=2)

cluster_0  \
Component                    Property    Unit                            
Biogas purchase              TAC         [1e9 Euro]            0.19243   
                             commodCosts [1e9 Euro/a]          0.19243   
                             operation   [GW_biogas_LHV*h/a]   3557.59   
CO2 to enviroment            operation   [Mio. t CO2/h*h/a]          0   
Electricity demand           operation   [GW_el*h/a]           30957.9   
Existing run-of-river plants TAC         [1e9 Euro]                NaN   
                             capacity    [GW_el]                   NaN   
                             operation   [GW_el*h/a]               NaN   
                             opexCap     [1e9 Euro/a]              NaN   
Hydrogen demand              operation   [GW_H2_LHV*h/a]       4765.07   
Natural gas purchase         TAC         [1e9 Euro]                  0   
                             commodCosts [1e9 Euro/a]                0   
                             operation   [GW_CH4_LHV*h/a]            0   
PV                           TAC         [1e9 Euro]            1.32784   
                             capacity    [GW_el]               17.9701   
                             capexCap    [1e9 Euro/a]          1.09422   
                             invest      [1e9 Euro]            11.6806   
                             operation   [GW_el*h/a]           11832.9   
                             opexCap     [1e9 Euro/a]         0.233612   
Wind (offshore)              TAC         [1e9 Euro]            4.51415   
                             capacity    [GW_el]                16.107   
                             capexCap    [1e9 Euro/a]          3.77323   
                             invest      [1e9 Euro]            37.0461   
                             operation   [GW_el*h/a]             50124   
                             opexCap     [1e9 Euro/a]         0.740923   
Wind (onshore)               TAC         [1e9 Euro]            7.35047   
                             capacity    [GW_el]                54.839   
                             capexCap    [1e9 Euro/a]          6.14402   
                             invest      [1e9 Euro]            60.3229   
                             operation   [GW_el*h/a]            103450   
                             opexCap     [1e9 Euro/a]          1.20646   

                                                             cluster_1  \
Component                    Property    Unit                            
Biogas purchase              TAC         [1e9 Euro]           0.240188   
                             commodCosts [1e9 Euro/a]         0.240188   
                             operation   [GW_biogas_LHV*h/a]   4440.52   
CO2 to enviroment            operation   [Mio. t CO2/h*h/a]    3.69324   
Electricity demand           operation   [GW_el*h/a]            111621   
Existing run-of-river plants TAC         [1e9 Euro]           0.113235   
                             capacity    [GW_el]              0.544397   
                             operation   [GW_el*h/a]           2668.85   
                             opexCap     [1e9 Euro/a]         0.113235   
Hydrogen demand              operation   [GW_H2_LHV*h/a]       13481.3   
Natural gas purchase         TAC         [1e9 Euro]            0.60819   
                             commodCosts [1e9 Euro/a]          0.60819   
                             operation   [GW_CH4_LHV*h/a]      18374.3   
PV                           TAC         [1e9 Euro]            1.90692   
                             capacity    [GW_el]               25.8072   
                             capexCap    [1e9 Euro/a]          1.57143   
                             invest      [1e9 Euro]            16.7747   
                             operation   [GW_el*h/a]           24922.2   
                             opexCap     [1e9 Euro/a]         0.335494   
Wind (offshore)              TAC         [1e9 Euro]             

### Conversion

In [34]:
esM.getOptimizationSummary("ConversionModeling", outputLevel=2)

cluster_0  cluster_1  \
Component                      Property  Unit                                 
Electroylzers                  TAC       [1e9 Euro/a]    1.78042          0   
                               capacity  [GW_el]         20.4612          0   
                               capexCap  [1e9 Euro/a]    1.52466          0   
                               invest    [1e9 Euro]      10.2306          0   
                               operation [GW_el*h/a]     96800.2          0   
                               opexCap   [1e9 Euro/a]   0.255765          0   
Existing CCGT plants (methane) TAC       [1e9 Euro/a]        NaN  0.0357861   
                               capacity  [GW_el]             NaN     1.7041   
                               operation [GW_el*h/a]         NaN      11484   
                               opexCap   [1e9 Euro/a]        NaN  0.0357861   
New CCGT plants (biogas)       TAC       [1e9 Euro/a]   0.049394  0.0446187   
                               capacity  [GW_el]        0.603867   0.545487   
                               capexCap  [1e9 Euro/a]  0.0367128  0.0331635   
                               invest    [1e9 Euro]     0.422707   0.381841   
                               operation [GW_el*h/a]     2259.07    2819.73   
                               opexCap   [1e9 Euro/a]  0.0126812  0.0114552   
New CCGT plants (hydrogen)     TAC       [1e9 Euro/a]          0          0   
                               capacity  [GW_el]               0          0   
                               capexCap  [1e9 Euro/a]          0          0   
                               invest    [1e9 Euro]            0          0   
                               operation [GW_el*h/a]           0          0   
                               opexCap   [1e9 Euro/a]          0          0   

                                                       cluster_2  cluster_3  \
Component                      Property  Unit                                 
Electroylzers                  TAC       [1e9 Euro/a]          0          0   
                               capacity  [GW_el]               0          0   
                               capexCap  [1e9 Euro/a]          0          0   
                               invest    [1e9 Euro]            0          0   
                               operation [GW_el*h/a]           0          0   
                               opexCap   [1e9 Euro/a]          0          0   
Existing CCGT plants (methane) TAC       [1e9 Euro/a]    0.06447        NaN   
                               capacity  [GW_el]            3.07        NaN   
                               operation [GW_el*h/a]     20775.3        NaN   
                               opexCap   [1e9 Euro/a]    0.06447        NaN   
New CCGT plants (biogas)       TAC       [1e9 Euro/a]  0.0433331  0.0682326   
                               capacity  [GW_el]         0.52977   0.834179   
                               capexCap  [1e9 Euro/a]  0.0322079  0.0507149   
                               invest    [1e9 Euro]     0.370839   0.583925   
                               operation [GW_el*h/a]     2663.15    4237.63   
                               opexCap   [1e9 Euro/a]  0.0111252  0.0175178   
New CCGT plants (hydrogen)     TAC       [1e9 Euro/a]   0.564553   0.582722   
                               capacity  [GW_el]         5.56147    5.74045   
                               capexCap  [1e9 Euro/a]   0.447762   0.462172   
                               invest    [1e9 Euro]      5.15549     5.3214   
                               operation [GW_el*h/a]     9443.38    9261.27   
                               opexCap   [1e9 Euro/a]   0.116791   0.120549   

                                                       cluster_4  cluster_5  \
Component                      Property  Unit                                 
Electroylzers                  TAC       [1e9 Euro/a]   0.716361          0   
                        

### Storage

In [35]:
esM.getOptimizationSummary("StorageModeling", outputLevel=2)

cluster_0  \
Component               Property           Unit                         
Li-ion batteries        TAC                [1e9 Euro]        0.386205   
                        capacity           [GW_el*h]          22.9845   
                        capexCap           [1e9 Euro/a]      0.340236   
                        invest             [1e9 Euro]         3.47066   
                        operationCharge    [GW_el*h/a]        7997.75   
                        operationDischarge [GW_el*h/a]        7213.18   
                        opexCap            [1e9 Euro/a]      0.045969   
Pumped hydro storage    TAC                [1e9 Euro]             NaN   
                        capacity           [GW_el*h]              NaN   
                        operationCharge    [GW_el*h/a]            NaN   
                        operationDischarge [GW_el*h/a]            NaN   
                        opexCap            [1e9 Euro/a]           NaN   
Salt caverns (hydrogen) TAC                [1e9 Euro]        0.716143   
                        capacity           [GW_H2_LHV*h]      1235.22   
                        capexCap           [1e9 Euro/a]     0.0120693   
                        invest             [1e9 Euro]        0.135874   
                        operationCharge    [GW_H2_LHV*h/a]    16708.7   
                        operationDischarge [GW_H2_LHV*h/a]    16708.7   
                        opexCap            [1e9 Euro/a]      0.704073   

                                                             cluster_1  \
Component               Property           Unit                          
Li-ion batteries        TAC                [1e9 Euro]         0.185034   
                        capacity           [GW_el*h]           11.0121   
                        capexCap           [1e9 Euro/a]        0.16301   
                        invest             [1e9 Euro]          1.66282   
                        operationCharge    [GW_el*h/a]         3379.08   
                        operationDischarge [GW_el*h/a]         3048.37   
                        opexCap            [1e9 Euro/a]      0.0220242   
Pumped hydro storage    TAC                [1e9 Euro]       0.00137868   
                        capacity           [GW_el*h]             9.011   
                        operationCharge    [GW_el*h/a]         3779.57   
                        operationDischarge [GW_el*h/a]          2926.8   
                        opexCap            [1e9 Euro/a]     0.00137868   
Salt caverns (hydrogen) TAC                [1e9 Euro]              NaN   
                        capacity           [GW_H2_LHV*h]           NaN   
                        capexCap           [1e9 Euro/a]            NaN   
                        invest             [1e9 Euro]              NaN   
                        operationCharge    [GW_H2_LHV*h/a]         NaN   
                        operationDischarge [GW_H2_LHV*h/a]         NaN   
                        opexCap            [1e9 Euro/a]            NaN   

                                                             cluster_2  \
Component               Property           Unit                          
Li-ion batteries        TAC                [1e9 Euro]         0.163705   
                        capacity           [GW_el*h]           9.74269   
                        capexCap           [1e9 Euro/a]        0.14422   
                        invest             [1e9 Euro]          1.47115   
                        operationCharge    [GW_el*h/a]         3176.24   
                        operationDischarge [GW_el*h/a]          2865.4   
                        opexCap            [1e9 Euro/a]      0.0194854   
Pumped hydro storage    TAC                [1e9 Euro]       0.00126455   
                        capacity           [GW_el*h]             8.265   
                        operationCharge    [GW_el*h/a]         3417.28   
                        operationDischarge [GW_el*h/a]         2646.26   
         

In [36]:
esM._componentModelingDict["StorageModeling"]._stateOfChargeOperationVariablesOptimum 

0            1            2     \
Li-ion batteries        cluster_0    22.961180    22.961180    22.961180   
                        cluster_1     1.255493     1.255493     1.255493   
                        cluster_2     2.251438     2.251438     2.251438   
                        cluster_3     1.808472     1.808472     1.808472   
                        cluster_4    48.351181    48.551858    49.778402   
                        cluster_5     2.476946     2.476946     2.476946   
                        cluster_6     3.274185     3.274185     3.274185   
                        cluster_7     2.369988     2.369988     2.369988   
Pumped hydro storage    cluster_1     0.000000     1.268749     2.537491   
                        cluster_2     0.000000     0.000000     1.163712   
                        cluster_3     0.000000     0.000000     0.000000   
                        cluster_4     0.000000     0.547571     1.095140   
                        cluster_5     0.000000     0.000000     0.440621   
                        cluster_6     0.000000     0.000000     1.374912   
                        cluster_7     0.000000     1.110486     1.112725   
Salt caverns (hydrogen) cluster_0  1108.744787  1110.642039  1112.588623   
                        cluster_2  4554.316427  4560.772770  4567.638888   
                        cluster_3  3329.744018  3332.877208  3336.172554   
                        cluster_4  7115.155426  7121.576057  7128.066244   
                        cluster_5   226.866660   227.450228   228.033795   
Salt caverns (methane)  cluster_0     0.000000     0.000000     0.000000   
                        cluster_2     0.000000     0.000000     0.000000   
                        cluster_3     0.000000     0.000000     0.000000   
                        cluster_4     0.000000     0.000000     0.000000   
                        cluster_5     0.000000     0.000000     0.000000   

                                          3            4            5     \
Li-ion batteries        cluster_0    22.984489    22.984488    22.984487   
                        cluster_1     1.255493     1.255493     1.255493   
                        cluster_2     2.251438     2.251438     2.251438   
                        cluster_3     1.808472     1.808472     1.808472   
                        cluster_4    50.423097    50.423010    45.457622   
                        cluster_5     2.476946     3.176369     4.785214   
                        cluster_6     3.274185     3.274185     9.022050   
                        cluster_7     2.369988     4.268627     4.268547   
Pumped hydro storage    cluster_1     3.178389     3.178373     3.178356   
                        cluster_2     1.163706     2.327412     2.327400   
                        cluster_3     2.762496     4.329625     4.329602   
                        cluster_4     1.642705     1.539266     1.539258   
                        cluster_5     1.006353     1.006348     1.006342   
                        cluster_6     1.374905     1.374898     1.374890   
                        cluster_7     2.605904     4.099074     4.099053   
Salt caverns (hydrogen) cluster_0  1114.551652  1116.504563  1118.435970   
                        cluster_2  4574.641605  4581.560275  4588.300322   
                        cluster_3  3339.521954  3342.838095  3346.667119   
                        cluster_4  7134.579617  7141.078723  7147.547511   
                        cluster_5   228.617362   229.200929   229.200929   
Salt caverns (methane)  cluster_0     0.000000     0.000000     0.000000   
                        cluster_2     0.000000     0.000000     0.000000   
                        cluster_3     0.000000     0.000000     0.000000   
                        cluster_4     0.000000     0.000000     0.000000   
                        cluster_5     0.000000     0.000000     0.000000   

                                          6            7            8     \
Li-ion batte

### Transmission

In [37]:
esM.getOptimizationSummary("TransmissionModeling", outputLevel=2)

cluster_0  \
Component           Property     Unit            Location                 
AC cables           capacity     [GW_el]         cluster_0          NaN   
                                                 cluster_1          NaN   
                                                 cluster_2          NaN   
                                                 cluster_3      2.39577   
                                                 cluster_4      3.26266   
                                                 cluster_5      2.39577   
                                                 cluster_6          NaN   
                                                 cluster_7          NaN   
                    operation    [GW_el*h/a]     cluster_0          NaN   
                                                 cluster_1          NaN   
                                                 cluster_2          NaN   
                                                 cluster_3      4043.16   
                                                 cluster_4      13376.3   
                                                 cluster_5      3641.35   
                                                 cluster_6          NaN   
                                                 cluster_7          NaN   
DC cables           capacity     [GW_el]         cluster_0          NaN   
                                                 cluster_1            2   
                                                 cluster_2          NaN   
                                                 cluster_3          NaN   
                                                 cluster_4          NaN   
                                                 cluster_7            2   
                    operation    [GW_el*h/a]     cluster_0          NaN   
                                                 cluster_1      1527.45   
                                                 cluster_2          NaN   
                                                 cluster_3          NaN   
                                                 cluster_4          NaN   
                                                 cluster_7      1245.95   
Pipeline (hydrogen) TAC          [1e9 Euro/a]    cluster_0            0   
                                                 cluster_1            0   
...                                                                 ...   
                    capexIfBuilt [1e9 Euro/a]    cluster_1            0   
                                                 cluster_2            0   
                                                 cluster_3            0   
                                                 cluster_4  1.38369e-05   
                                                 cluster_5            0   
                                                 cluster_6            0   
                                                 cluster_7            0   
                    invest       [1e9 Euro]      cluster_0            0   
                                                 cluster_1            0   
                                                 cluster_2            0   
                                                 cluster_3            0   
                                                 cluster_4   0.00125458   
                                                 cluster_5            0   
                                                 cluster_6            0   
                                                 cluster_7            0   
                    isBuilt      [-]             cluster_0          NaN   
                                                 cluster_1          NaN   
                                                 cluster_2          NaN   
                                                 cluster_3          NaN   
                                                 cluster_4            1   
                                                 cluster_5          NaN   
            

In [39]:
esM._componentModelingDict["TransmissionModeling"]._operationVariablesOptimum.head()

0         1         2         3     \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  1.871848  2.386227  2.472146  2.118043   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  0.000000  2.117540  1.554303  2.270486   
                    cluster_3  3.705039  0.000000  4.397688  0.000000   

                                   4         5         6         7     \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  0.572120  0.000000  0.000000  0.000000   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  1.727292  2.241318  2.240932  2.339679   
                    cluster_3  0.000000  0.000000  0.000000  0.000000   

                                   8         9       ...         8750  \
AC cables cluster_0 cluster_3  2.395773  2.395773    ...     2.395773   
                    cluster_4  0.000000  0.000000    ...     1.844226   
                    cluster_5  2.395773  2.395773    ...     2.395773   
          cluster_1 cluster_2  2.796092  2.485289    ...     2.633084   
                    cluster_3  0.000000  0.000000    ...     0.000000   

                                   8751      8752      8753      8754  \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  2.718049  2.916154  0.000000  3.262664   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  0.000000  2.142334  1.226603  2.558835   
                    cluster_3  0.000000  0.000000  0.000000  0.000000   

                                   8755      8756      8757      8758  \
AC cables cluster_0 cluster_3  2.395773  2.395773  2.395773  2.395773   
                    cluster_4  3.262664  0.000000  3.262664  0.000000   
                    cluster_5  2.395773  2.395773  2.395773  2.395773   
          cluster_1 cluster_2  2.403483  0.000000  0.000000  0.000000   
                    cluster_3  0.000000  0.000000  2.808756  1.760348   

                                   8759  
AC cables cluster_0 cluster_3  2.395773  
                    cluster_4  1.909219  
                    cluster_5  2.395773  
          cluster_1 cluster_2  0.000000  
                    cluster_3  1.690629  

[5 rows x 8760 columns]